# Importação de Bibliotecas

In [1]:
# Python versão 3.8.8
# Windows 10

# biblioteca para interação com o sistema
import os
import shutil

# biblioteca para manipulação de dados
import pandas as pd # versão 1.3.2
import numpy as np

# biblioteca para manipulação de banco de dados sqlite
import sqlite3 # versão 2.6.0
from sqlite3 import Error

import warnings
warnings.filterwarnings('ignore', category=UserWarning, module='openpyxl')

from zipfile import ZipFile

## Variáveis Globais

In [2]:
path_atual = os.path.dirname(os.path.realpath('__file__')) + "\\" # diretório onde se encontra o script
path_dados = path_atual + "Microdados_Covid\\"
# nome_bd_sp = "COVID_SP_DW.sqlite"
nome_bd_mg = "COVID_MG_DW.sqlite"

# Leitura de Dados

In [3]:
ibge_populacao = pd.read_excel(path_dados + "Estimativa_Populacao_IBGE.xls", sheet_name = "Municípios", 
                               skiprows=1, skipfooter = 22)

In [4]:
zip_casos = ZipFile(path_dados + "base_de_dados_2010_2018_xls.zip")

arquivo = 'PIB dos Municípios - base de dados 2010-2018.xls'
ibge_pib = pd.read_excel(zip_casos.open(arquivo), usecols = "A,E,G,AN", skiprows = list(np.arange(1, 44546)))

In [5]:
data_sistema = pd.read_csv(path_dados + "CSV_Sistemas.zip", sep = ";", low_memory=False)

zip_casos = ZipFile(path_dados + "CSV_Laboratorios.zip")

arquivo = 'CSV_Laboratorios_1.csv'
data_laboratorios_1 = pd.read_csv(zip_casos.open(arquivo), sep=';', low_memory=False)

arquivo = 'CSV_Laboratorios_2.csv'
data_laboratorios_2 = pd.read_csv(zip_casos.open(arquivo), sep=';', low_memory=False)

data_casos_mg = pd.concat([data_sistema, data_laboratorios_1, data_laboratorios_2]).reset_index(drop = True)

# Funções

### Função para criar uma nova conexão ao Banco de Dados

In [6]:
def nova_conexao(path, nome_bd):

    conec_bd = None
    try:
        conec_bd = sqlite3.connect(path + nome_bd)

    except Error as erro:
        print(erro)
        return 0
    
    finally:
        if conec_bd:
            conec_bd.close()
    return 1

### Funções inserir uma tabela no Banco de Dados a partir de um DataFrame

In [7]:
def verifica_tabela_existe(cursor, nome_tabela):
    
    cursor.execute(''' SELECT name FROM sqlite_master WHERE type='table' AND name='{}' '''.format(nome_tabela))
    
    if cursor.fetchone():
        return 1
    else:
        return 0

In [8]:
def coverte_tipo_sqlite(string_tipo):  
        
    if string_tipo in ["int32", "int64", "bool"]:
            tipo_sql = "integer"
            
    elif string_tipo == "float64":
            tipo_sql = "real"           
    else:
        tipo_sql = "text"
    
    return tipo_sql
    

In [9]:
def insere_tabelas_banco(data, cursor, nome_tabela):

    tabela_existe = verifica_tabela_existe(cursor, nome_tabela)

    if not tabela_existe:


        # Transforma o nome da coluna seguido por seu tipo em apenas uma string para cada coluna
        # Logo após concatena essas strings utilizando a vírgula como separador
        lista_colunas = list(data.columns)
        
        novas_colunas = []
        for coluna in lista_colunas:
            coluna = coluna + " " + coverte_tipo_sqlite(str(data[coluna].dtype)) 
            novas_colunas.append(coluna)

        nomes_colunas = ", ".join(novas_colunas)         

        print("Criando tabela {} no banco {}...".format(nome_tabela, nome_bd), end="\r", flush=True)

        # Cria a tabela no banco de dados sqlite passando o nome da tabela, o nome das colunas e seus tipos
        comando = 'CREATE TABLE ' + nome_tabela + ' (' + nomes_colunas + ')'
        cursor.execute('' + comando + '')

        # Importa o dataframe com microdados referentes ao no para a tabela no banco de dados
        data.to_sql(nome_tabela, conec_bd, if_exists='append', index = False)
        conec_bd.commit()

        print(" "*70, end="\r", flush=True)
        print("Tabela {} criada com sucesso no banco {}.".format(nome_tabela, nome_bd), end="\n\n")

    # Se a tabela já existe, nada acontece
    else:
        print("Tabela {} já existe no banco de dados {}.".format(nome_tabela, nome_bd), end="\n\n")

# Minas Gerais

## Conexão Banco de Dados

In [10]:
if (nova_conexao(path_atual,  nome_bd_mg)):
    conec_bd = sqlite3.connect(path_atual + nome_bd_mg)
    cursor = conec_bd.cursor()
    print("Conexão ao banco '{}' criada com sucesso.".format(nome_bd_mg))
    conec_bd.commit()

Conexão ao banco 'COVID_MG_DW.sqlite' criada com sucesso.


## Manipulacao de Dados

### Remoção de Colunas

In [11]:
data_casos_mg = data_casos_mg.drop(columns = ['DATA_EVOLUCAO','ETNIA','COMORBIDADE','DATA_ATUALIZACAO','RACA',
                                             'CLASSIFICACAO_CASO', 'ORIGEM_DA_INFORMACAO'])

### Remoção de Nulos

In [12]:
data_casos_mg.SEXO = np.where(data_casos_mg.SEXO == "NAO INFORMADO", np.nan, data_casos_mg.SEXO)
data_casos_mg.INTERNACAO = np.where(data_casos_mg.INTERNACAO == "NAO INFORMADO", np.nan, data_casos_mg.INTERNACAO)
data_casos_mg.UTI = np.where(data_casos_mg.UTI == "NAO INFORMADO", np.nan, data_casos_mg.UTI)

data_casos_mg = data_casos_mg.dropna(subset=['CODIGO', 'SEXO', 'DATA_NOTIFICACAO','INTERNACAO', 'UTI', 'DATA_1_SINTOMA','MICRO']).reset_index(drop = True)

In [13]:
data_casos_mg = data_casos_mg[data_casos_mg.IDADE <= 109].reset_index(drop = True)

### Criação de Colunas

In [14]:
data_casos_mg["DATA_NOTIFICACAO"] = pd.to_datetime(data_casos_mg["DATA_NOTIFICACAO"], format='%d/%m/%Y')
data_casos_mg["DATA_1_SINTOMA"] = pd.to_datetime(data_casos_mg["DATA_1_SINTOMA"], format='%d/%m/%Y')
data_casos_mg['DIAS_PRA_NOTIFICACAO'] = (data_casos_mg["DATA_NOTIFICACAO"] - data_casos_mg["DATA_1_SINTOMA"]).dt.days

In [15]:
data_casos_mg = data_casos_mg[data_casos_mg.DIAS_PRA_NOTIFICACAO >= 0].reset_index(drop = True)

### Renomeação de Colunas

In [16]:
data_casos_mg = data_casos_mg.rename(columns = {"CODIGO":"ID_MUNICIPIO",
                                                "ID": "ID_PESSOA",
                                                "SEXO": "GENERO",
                                                "DATA_NOTIFICACAO":"ID_TEMPO"})

### Conversão Tipos das Colunas

In [17]:
data_casos_mg.dtypes

ID_PESSOA                        int64
URS                             object
MICRO                           object
MACRO                           object
ID_TEMPO                datetime64[ns]
GENERO                          object
IDADE                          float64
FAIXA_ETARIA                    object
MUNICIPIO_RESIDENCIA            object
ID_MUNICIPIO                   float64
EVOLUCAO                        object
DATA_1_SINTOMA          datetime64[ns]
INTERNACAO                      object
UTI                             object
DIAS_PRA_NOTIFICACAO             int64
dtype: object

In [18]:
data_casos_mg.IDADE = data_casos_mg.IDADE.astype(int)
data_casos_mg.ID_MUNICIPIO = data_casos_mg.ID_MUNICIPIO.astype(int)

## Alimentação do Banco de Dados

In [19]:
nome_bd = nome_bd_mg

### Criação Tabela Dimensão Paciente

In [20]:
dim_paciente_mg = data_casos_mg[['ID_PESSOA', 'GENERO', 'IDADE', 'FAIXA_ETARIA']]

In [21]:
insere_tabelas_banco(dim_paciente_mg, cursor, "Dim_Paciente")

Tabela Dim_Paciente criada com sucesso no banco COVID_MG_DW.sqlite.   



### Criação Tabela Dimensão Tempo

In [22]:
dim_tempo = data_casos_mg[["ID_TEMPO"]].drop_duplicates().reset_index(drop = True)

In [23]:
dim_tempo["Dia"] = dim_tempo["ID_TEMPO"].dt.day
dim_tempo["Mes"] = dim_tempo["ID_TEMPO"].dt.month
dim_tempo["Ano"] = dim_tempo["ID_TEMPO"].dt.year

In [24]:
dim_tempo = dim_tempo.sort_values(by = "ID_TEMPO").reset_index(drop = "True")

In [25]:
insere_tabelas_banco(dim_tempo, cursor, "Dim_Tempo")

Tabela Dim_Tempo criada com sucesso no banco COVID_MG_DW.sqlite.      



### Criação Tabela Dimensão Cidade

In [26]:
dim_cidade = data_casos_mg[["ID_MUNICIPIO", "MUNICIPIO_RESIDENCIA", "MICRO", "URS", "MACRO"]]
dim_cidade = dim_cidade.drop_duplicates(subset=["ID_MUNICIPIO"]).reset_index(drop = True)

In [27]:
ibge_populacao["COD. MUNIC"] = ibge_populacao["COD. MUNIC"].astype(str).str[:-1].str.zfill(4)
ibge_populacao["ID_MUNICIPIO"] = ibge_populacao["COD. UF"].astype(str)  + ibge_populacao["COD. MUNIC"] 

In [28]:
ibge_populacao["ID_MUNICIPIO"] = ibge_populacao["ID_MUNICIPIO"].astype(int)

In [29]:
ibge_populacao = ibge_populacao.rename(columns = {"POPULAÇÃO ESTIMADA":"POPULAÇÃO_ESTIMADA"})

In [30]:
dim_cidade["ID_MUNICIPIO"] = dim_cidade["ID_MUNICIPIO"].astype(int)

In [31]:
dim_cidade = pd.merge(dim_cidade, ibge_populacao[["ID_MUNICIPIO", "POPULAÇÃO_ESTIMADA"]], on="ID_MUNICIPIO")

In [32]:
ibge_pib = ibge_pib.rename(columns={
    "Sigla da Unidade da Federação": "UF", 
    "Código do Município": "ID_MUNICIPIO",
    "Produto Interno Bruto per capita, \na preços correntes\n(R$ 1,00)": "PIB_PER_CAPITA"})

In [33]:
ibge_pib["ID_MUNICIPIO"] = ibge_pib["ID_MUNICIPIO"].astype(str).str[:-1].astype(int)

In [34]:
dim_cidade = pd.merge(dim_cidade, ibge_pib[["ID_MUNICIPIO", "PIB_PER_CAPITA"]], on="ID_MUNICIPIO")

In [35]:
dim_cidade["POPULAÇÃO_ESTIMADA"] = dim_cidade["POPULAÇÃO_ESTIMADA"].astype(int)

In [36]:
insere_tabelas_banco(dim_cidade, cursor, "Dim_Municipio")

Tabela Dim_Municipio criada com sucesso no banco COVID_MG_DW.sqlite.  



### Tabela Dimensão Tratamento

In [37]:
dim_tratamento = data_casos_mg[["ID_PESSOA", 'DIAS_PRA_NOTIFICACAO','EVOLUCAO', 'INTERNACAO', 'UTI']]

In [38]:
insere_tabelas_banco(dim_tratamento, cursor, "Dim_Status_Tratamento")

Tabela Dim_Status_Tratamento criada com sucesso no banco COVID_MG_DW.sqlite.



### Criação Tabela Fato

In [39]:
tabela_fato = data_casos_mg[['ID_PESSOA', 'ID_MUNICIPIO','ID_TEMPO']]

In [40]:
insere_tabelas_banco(tabela_fato, cursor, "Fato_Internacao")

Tabela Fato_Internacao criada com sucesso no banco COVID_MG_DW.sqlite.



## Fechamento da Conexão com o Banco de dados

In [41]:
conec_bd.close()